In [1]:
#notebook to clean/modify data to a suitable format
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import math
#import map
import requests
countries_map = requests.get('https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson').json()
df = pd.read_csv('data_clean.csv')
#convert dates to pandas date object
import datetime
#choose template
import plotly.io as pio
pio.templates.default="seaborn"
#take colourmaps from matplotlib
import matplotlib
import matplotlib.cm as cm
import colorcet as cc
#df['date'] = pd.to_datetime(arg=df['date'],yearfirst=True)
df

,position,track_name,artist,streams,id,date,region,nationality
0,1,Reggaetón Lento (Bailemos),CNCO,19272,3AEZUABDXNtecAOSC1qTfo,2017-01-01,ECU,USA
1,43,Tan Fácil,CNCO,4205,1TxgZRcy3kZ7JaEp88MDbC,2017-01-01,ECU,USA
2,81,Quisiera,CNCO,2347,4iPV0gcc8keTFl6qxqP73M,2017-01-01,ECU,USA
3,119,Para Enamorarte,CNCO,1791,3B21fRFqPLmTd9zAlK0I8A,2017-01-01,ECU,USA
4,153,Tan Fácil - Urban Remix,CNCO,1454,6ZvmnulA6jJY3lvveuCz77,2017-01-01,ECU,USA
...,...,...,...,...,...,...,...,...
3007235,190,終止戀愛,E-Kids,2211,5IqINoJbcsew286bOMeJ7N,2018-01-02,HKG,CHN
3007236,163,終止戀愛,E-Kids,2489,5IqINoJbcsew286bOMeJ7N,2018-01-03,HKG,CHN
3007237,172,終止戀愛,E-Kids,2399,5IqINoJbcsew286bOMeJ7N,2018-01-04,HKG,CHN
3007238,166,終止戀愛,E-Kids,2468,5IqINoJbcsew286bOMeJ7N,2018-01-05,HKG,CHN


In [56]:
df['date'].max()

Timestamp('2018-01-09 00:00:00')

In [57]:
np.linspace(0,256,len(df_cur_day),dtype=int)

array([  0,   4,   9,  14,  19,  24,  29,  34,  39,  44,  49,  54,  59,
        64,  68,  73,  78,  83,  88,  93,  98, 103, 108, 113, 118, 123,
       128, 132, 137, 142, 147, 152, 157, 162, 167, 172, 177, 182, 187,
       192, 196, 201, 206, 211, 216, 221, 226, 231, 236, 241, 246, 251,
       256])

In [6]:
#create a colourmap (dont need anymore)
cmap = cm.jet(np.linspace(0,10,len(df_cur_day)))
cmap_hex = []
for col in cmap:
    cmap_hex.append(matplotlib.colors.to_hex(col))

In [59]:
np.linspace(0,255,len(df_cur_day),dtype=int)

array([  0,   4,   9,  14,  19,  24,  29,  34,  39,  44,  49,  53,  58,
        63,  68,  73,  78,  83,  88,  93,  98, 102, 107, 112, 117, 122,
       127, 132, 137, 142, 147, 152, 156, 161, 166, 171, 176, 181, 186,
       191, 196, 201, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250,
       255])

In [9]:
cmap = []
for val in np.linspace(0,255,len(df_cur_day),dtype=int):
    cmap.append(cc.CET_R2[val])
    #print(val)
cmap = cc.glasbey_light
#https://colorcet.holoviz.org/user_guide/index.html colours

In [70]:
#plot the most popular song in each country on a certain date
#choose date/artist
date = '2017-01-01'
df_small = df[df['date'] == date]
df_cur_day = pd.DataFrame(columns=df_small.columns)
for reg in set(df_small['region']):
    df_cur_day = df_cur_day.append(df_small[(df_small['position'] == 1) & (df_small['region'] == reg)])
print(date)
template = '<b>%{hovertext}</b><br><br>Region: %{customdata[0]}<br>Artist: %{customdata[1]}<br>Streams: %{customdata[2]}<extra></extra>'

#(use this for streams) df_cur_day['streams'] = df_cur_day['streams'].astype(int)
#plot data
fig = px.choropleth(df_cur_day, locations="region",
                    color="id", # lifeExp is a column of gapminder
                    hover_name="track_name", # column to add to hover information
                    #color_discrete_sequence=px.colors.qualitative.Set3,
                    color_discrete_sequence=cmap,#px.colors.cyclical.HSV,
                    labels={"id" : "Song name"},
                    hover_data={'region': True, 'artist': True,'streams': True, 'id': False})
fig.update_layout(showlegend = True)
fig.update_traces(hovertemplate=template)
for i in range(len(fig.data)):
    fig.data[i].name = fig.data[i].hovertext[0]
fig.show()

2017-01-01


In [152]:
#these countries do not have the full 371 day range of number one songs?
for reg in set(df['region']):
    if len(df[(df['position'] == 1) & (df['region'] == reg)]['date']) != 371:
        print(reg + ' - ' + str(len(df[(df['position'] == 1) & (df['region'] == reg)]['date'])))

EST - 265
LTU - 302
SVK - 367
MYS - 370
LUX - 217
LVA - 348


In [48]:
df_small[df_small['streams'] == df_small[df_small['region'] == reg].streams.max()] #taking the highest streamed song in a particular region

,position,track_name,artist,streams,id,date,region
1174714,1,Shape of You,Ed Sheeran,24325,7qiZfU4dY1lWllzX7mPBI3,2017-12-15,JPN


In [437]:
df_global_third_day = pd.DataFrame(columns=df_global.columns)
for i in range(10):
    df_global_third_day = df_global_third_day.append(df_global[df_global['position'] <= 10][i::40])
df_global_third_day['date']

0        2017-01-01
800      2017-01-05
1600     2017-01-09
2400     2017-01-13
3200     2017-01-17
            ...    
70409    2017-12-24
71209    2017-12-28
72009    2018-01-01
72809    2018-01-05
73609    2018-01-09
Name: date, Length: 930, dtype: object

In [440]:
#gives the top streamed songs globally    
df_global = pd.read_csv('data_global.csv')
df_fig = df_global[df_global['position'] <= 10]
#df_fig = df_global_third_day
fig = px.bar(df_fig,x='streams',y='position',animation_frame='date',animation_group='id',orientation='h',hover_data={'track_name': True}, color='id',color_discrete_sequence=cmap,title='10 Most Popular Songs Each Day',labels={'id' : 'Track name'})
fig.update_layout(transition={'easing' : 'exp','duration' : 10000}, yaxis={'tickmode' : 'linear'})
print('here')
#fig.frames[0].data[0]['name'] = df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name'].iloc[0]
#change names from ID to track name

for frame in range(len(fig.frames)):
    for name in range(len(fig.frames[frame].data)):
        #track_name = df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name'].iloc[0]
        track_name = fig.frames[frame].data[name]['customdata'][0][0]
        fig.frames[frame].data[name]['name'] = track_name
        #print(str(frame) + ',' + str(name))
#slow down animation
time = 5000
# fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = time - 1000
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = time
# fig.layout.updatemenus[0].buttons[1].args[1]['frame']['duration'] = time - 1000
# fig.layout.updatemenus[0].buttons[1].args[1]['transition']['duration'] = time
#fig.update_traces(name='s') 'id' : 'track_name'}
#fig.show()
#iplot(fig,show_link=True)
plot(fig)

here


'temp-plot.html'

In [363]:
fig.layout.updatemenus#[0].buttons[0].args[1]['transition']['duration']

(layout.Updatemenu({
     'buttons': [{'args': [None, {'frame': {'duration': 10000, 'redraw': True},
                           'mode': 'immediate', 'fromcurrent': True, 'transition':
                           {'duration': 10000, 'easing': 'linear'}}],
                  'label': '&#9654;',
                  'method': 'animate'},
                 {'args': [[None], {'frame': {'duration': 10000, 'redraw': True},
                           'mode': 'immediate', 'fromcurrent': True, 'transition':
                           {'duration': 10000, 'easing': 'linear'}}],
                  'label': '&#9724;',
                  'method': 'animate'}],
     'direction': 'left',
     'pad': {'r': 10, 't': 70},
     'showactive': False,
     'type': 'buttons',
     'x': 0.1,
     'xanchor': 'right',
     'y': 0,
     'yanchor': 'top'
 }),)

In [238]:
#testing version of above
#gives the top streamed songs globally    
df_global = pd.read_csv('data_global.csv')
df_global
fig = px.bar(df_global[df_global['position'] <= 10].head(600),x='streams',y='track_name',animation_frame='date',animation_group='id',orientation='h',hover_data={'track_name': True, 'id' : 'track_name'}, color='id',color_discrete_sequence=cmap,title='10 Most Popular Songs Each Day',labels={'id' : 'Track name'},category_orders={'track_name' : 'max descending'})
fig.update_layout(transition={'easing' : 'linear','duration' : 1000, 'ordering' : 'traces first'},updatemenus=[dict(buttons=list([dict(args=args,label=label,method=method)]))])#, yaxis={'tickmode' : 'array', 'tickvals': list(range(11)), 'ticktext' : df_global['track_name']})
#fig.show()
#iplot(fig,show_link=True)
plot(fig)

'temp-plot.html'

In [235]:
fig.frames[0].data[0]['name'] = df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name'].iloc[0]

#['hovertemplate']

In [258]:
fig.frames[0].data[0]['customdata'][0][0]

'Starboy'

In [ ]:
df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name']

In [243]:
for frame in len(fig.frames):
    for name in len(fig.frames[frame].data):
        track_name = df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name'].iloc[0]
        fig.frames[frame].data[name]['name'] = track_name

60

In [234]:
df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']['track_name'].iloc[0]

'Starboy'

In [25]:
fig.update_layout(yaxis={'t': 'a'})

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.YAxis: 't'

    Valid properties:
        anchor
            If set to an opposite-letter axis id (e.g. `x2`, `y`),
            this axis is bound to the corresponding opposite-letter
            axis. If set to "free", this axis' position is
            determined by `position`.
        automargin
            Determines whether long tick labels automatically grow
            the figure margins.
        autorange
            Determines whether or not the range of this axis is
            computed in relation to the input data. See `rangemode`
            for more info. If `range` is provided, then `autorange`
            is set to False.
        calendar
            Sets the calendar system to use for `range` and `tick0`
            if this is a date axis. This does not set the calendar
            for interpreting data on this axis, that's specified in
            the trace or via the global `layout.cal

In [161]:
a[1]['fromcurrent'] = False
a

(None,
 {'frame': {'duration': 500, 'redraw': True},
  'mode': 'immediate',
  'fromcurrent': False,
  'transition': {'duration': 500, 'easing': 'linear'}})

In [181]:
args = fig.layout.updatemenus[0]['buttons'][0]['args']
args[1]['fromcurrent'] = False
label = fig.layout.updatemenus[0]['buttons'][0]['label']
method = fig.layout.updatemenus[0]['buttons'][0]['method']

In [188]:
fig.layout.updatemenus[0]['buttons'][1]

layout.updatemenu.Button({
    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'mode':
             'immediate', 'fromcurrent': False, 'transition': {'duration': 500,
             'easing': 'linear'}}],
    'label': '&#9654;',
    'method': 'animate'
})

In [132]:
df[df['date'] == '2017-01-01']['id'].value_counts()

5aAx2yezTd8zXrkmtKl66Z    53
5MFzQMkrl1FOOng9tq6R9r    52
5knuzwU65gJK7IF5yJsuaW    52
1xznGGDReH1oQq0xzbwXa3    52
5GXAXm5YOmYT0kL5jHvYBt    52
                          ..
3CE1HoJJ0FVKo7JHsWIuvC     1
6KSrm7psP1KvqBRXcD42AS     1
4LeP389CNcbrQX8ZECZlPw     1
31L9yLXSj6LpCFupyMV6CR     1
15AN6WwAraTEuS0SGDl4Xr     1
Name: id, Length: 2361, dtype: int64

In [124]:
df[df['date'] == '2017-01-01'].groupby(by='id').sum().sort_values(by='streams',ascending=False)

,position,streams
id,,
5aAx2yezTd8zXrkmtKl66Z,421,3130581
7BKLCZ1jbUBVqRi2FVlTVw,539,3010132
4pdPtRcBmOSQDlJ3Fk945m,464,2540156
5knuzwU65gJK7IF5yJsuaW,926,2352144
1xznGGDReH1oQq0xzbwXa3,1010,2255673
...,...,...
30CenAPBR2VmH91r4Um32b,108,1020
7uTBq4YIpKyCBWMf1tGap2,195,1018
0NEj7uUWXkaDgKzMwnwSbh,73,1009


In [118]:
df[['track_name','id']]

,track_name,id
0,Reggaetón Lento (Bailemos),3AEZUABDXNtecAOSC1qTfo
1,Chantaje,6mICuAdrwEjh6Y6lroV2Kg
2,Otra Vez (feat. J Balvin),3QwBODjSEzelZyVjxPOHdq
3,Vente Pa' Ca,7DM4BPaS7uofFul3ywMe46
4,Safari,6rQSrBHf7HlZjtcMZ4S4bO
...,...,...
3367392,More Than You Know,71bBFbfn2OBK5QwUJSLS44
3367393,Issues,7vu0JkJh0ldukEYbTVcqd0
3367394,Tuo Fei Lun,0pmGleExiQyd0HxV5kAOwK
3367395,面具,58SUsgamkW6vFFq0YNKGrf


In [100]:
df[df['id'] == '5aAx2yezTd8zXrkmtKl66Z']

,position,track_name,artist,streams,id,date,region
18,19,Starboy,The Weeknd,7006,5aAx2yezTd8zXrkmtKl66Z,2017-01-01,ECU
214,15,Starboy,The Weeknd,7330,5aAx2yezTd8zXrkmtKl66Z,2017-01-02,ECU
410,11,Starboy,The Weeknd,8676,5aAx2yezTd8zXrkmtKl66Z,2017-01-03,ECU
610,11,Starboy,The Weeknd,8903,5aAx2yezTd8zXrkmtKl66Z,2017-01-04,ECU
811,12,Starboy,The Weeknd,8305,5aAx2yezTd8zXrkmtKl66Z,2017-01-05,ECU
...,...,...,...,...,...,...,...
3365552,156,Starboy,The Weeknd,2313,5aAx2yezTd8zXrkmtKl66Z,2017-12-31,HKG
3365778,182,Starboy,The Weeknd,1799,5aAx2yezTd8zXrkmtKl66Z,2018-01-01,HKG
3365987,191,Starboy,The Weeknd,2199,5aAx2yezTd8zXrkmtKl66Z,2018-01-02,HKG
3366181,185,Starboy,The Weeknd,2263,5aAx2yezTd8zXrkmtKl66Z,2018-01-03,HKG
